In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from nbainjuries import injury
from datetime import datetime, timedelta
import warnings
import duckdb
import os
import requests

cwd = os.path.abspath(os.getcwd()).replace("\\", "/")
if cwd.startswith("C:/Users/Rodolfo/"):
    RUN_LOCATION = "local"
else:
    RUN_LOCATION = "cloud"
time_offset = {"local": 3, "cloud": -5}

print("Current working dir:", cwd)
print("RUN_LOCATION =", RUN_LOCATION)

Current working dir: C:/Users/Rodolfo/Jupyter_files/FantasyBasketball/notebooks
RUN_LOCATION = local


In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
warnings.filterwarnings("ignore")

year = 2025
now = str((datetime.now() + timedelta(hours=time_offset[RUN_LOCATION]) + timedelta(hours=-3)).date())
print(f"Today's date:", now)

con = duckdb.connect(database=":memory:")
categories = ['PTS', 'AST', 'REB', 'PR', 'PA', 'RA', 'PRA', 'TPM', 'STL', 'BLK', 'STL_BLK']

folders = os.listdir('../tables/')
df = pd.DataFrame()
for yr in folders:
    df_temp = pd.read_csv(f"../tables/{yr}/season_gamelogs.csv")
    df_temp.insert(0, 'Season', int(yr))
    df = pd.concat([df, df_temp])
df['Date'] = pd.to_datetime(df.Date)
df = df.rename(columns={"TRB": "REB", "3PM": "TPM", "3PA": "TPA"})
df['STL_BLK'] = df.STL + df.BLK
df['PR'] = df.PTS + df.REB 
df['PA'] = df.PTS + df.AST
df['RA'] = df.REB + df.AST
df['PRA'] = df.PTS + df.REB + df.AST

# # CONTROL DATE TO GO BACK AND RELOAD HISTORICAL DATA
# df = df[(df.Date != now)]

Today's date: 2025-12-03


In [3]:
%run ./common_utils.ipynb

In [4]:
df_gms = pd.read_csv(f"../tables/{year}/nba_schedule.csv")
df_gms = df_gms[(df_gms.Date == now)]
tms_today = df_gms.AwayABV.tolist() + df_gms.HomeABV.tolist()
df_gms['gm_id'] = df_gms.AwayABV + "_" + df_gms.HomeABV
df_gms['gm_id2'] = df_gms.HomeABV + "_" + df_gms.AwayABV
gms_today = df_gms.gm_id.tolist() + df_gms.gm_id2.tolist()
df_gms = df_gms.drop(['gm_id', 'gm_id2'], axis=1)
display(df_gms)

,Date,StartTime_ET,AwayTeam,AwayABV,HomeTeam,HomeABV,Arena,AwayB2B,HomeB2B,rtrvd
314,2025-12-03,7:00p,Portland Trail Blazers,POR,Cleveland Cavaliers,CLE,Rocket Arena,1,0,0
315,2025-12-03,7:00p,Denver Nuggets,DEN,Indiana Pacers,IND,Gainbridge Fieldhouse,0,0,0
316,2025-12-03,7:00p,San Antonio Spurs,SAS,Orlando Magic,ORL,Kia Center,1,0,0
317,2025-12-03,7:30p,Los Angeles Clippers,LAC,Atlanta Hawks,ATL,State Farm Arena,0,0,0
318,2025-12-03,7:30p,Charlotte Hornets,CHO,New York Knicks,NYK,Madison Square Garden (IV),0,1,0
319,2025-12-03,8:00p,Brooklyn Nets,BRK,Chicago Bulls,CHI,United Center,0,0,0
320,2025-12-03,8:00p,Sacramento Kings,SAC,Houston Rockets,HOU,Toyota Center,0,0,0
321,2025-12-03,8:00p,Detroit Pistons,DET,Milwaukee Bucks,MIL,Fiserv Forum,0,0,0
322,2025-12-03,8:30p,Miami Heat,MIA,Dallas Mavericks,DAL,American Airlines Center,0,0,0


In [5]:
df_teams = pd.read_csv("../src/team_info_xref.csv")

df_inj = injury.get_reportdata(datetime.now() + timedelta(hours=time_offset[RUN_LOCATION]), return_df=True)
df_inj = df_inj.rename(columns={"Game Date": "Date", "Player Name": "Player", "Current Status": "Status"})
df_inj['Player'] = df_inj.Player.str.split(",").str[1] + " " + df_inj.Player.str.split(",").str[0]
df_inj['Player'] = df_inj['Player'].str.strip()
df_inj['Date'] = pd.to_datetime(df_inj['Date'])
df_inj = con.execute(f"""SELECT Date, ABV as Team, Player, Status, Reason FROM df_inj 
                         JOIN df_teams ON df_inj.Team = df_teams.Team
                         WHERE Date = '{now}' AND Status = 'Out'""").fetchdf()

df_best_out = con.execute(f"""SELECT *, RANK() OVER (PARTITION BY Pos ORDER BY Fpts DESC) as Off_Rk FROM
                              (SELECT Team, Player, Pos, AVG(Fpts) as Fpts FROM df 
                              WHERE Season = 2025 AND Fpts > 0 AND Pos != 'None'
                              GROUP BY Team, Player, Pos)""").fetchdf()
df_best_out = con.execute(f"""SELECT * EXCLUDE(t2.Team, t2.Player) FROM df_best_out t1 
                              JOIN df_inj t2 ON t1.Team = t2.Team AND t1.Player = t2.Player
                              ORDER BY Fpts DESC""").fetchdf()
display(df_best_out)

Validated Injury-Report_2025-12-03_03PM.


,Team,Player,Pos,Fpts,Off_Rk,Date,Status,Reason
0,SAS,Victor Wembanyama,C,37.833333,2,2025-12-03,Out,Injury/Illness - Left Calf; Strain
1,SAC,Domantas Sabonis,C,25.772727,6,2025-12-03,Out,Injury/Illness - Left Knee; Partial Meniscus Tear
2,ORL,Paolo Banchero,PF,25.125000,9,2025-12-03,Out,Injury/Illness - Left Groin; strain
3,POR,Jrue Holiday,PG,24.125000,11,2025-12-03,Out,Injury/Illness - Right Calf; Strain
4,SAS,Stephon Castle,PG,23.115385,14,2025-12-03,Out,Injury/Illness - Left Hip; Flexor Strain
5,ATL,Kristaps Porzingis,C,22.500000,14,2025-12-03,Out,Injury/Illness - N/A; Illness
6,CHI,Coby White,SG,21.400000,8,2025-12-03,Out,Injury/Illness - Left Calf; Strain
7,MIA,Norman Powell,SG,21.205882,9,2025-12-03,Out,Injury/Illness - Left Ankle; Sprain
8,NYK,OG Anunoby,PF,20.375000,14,2025-12-03,Out,Injury/Illness - Left Hamstring; Strain
9,ATL,Trae Young,PG,19.600000,21,2025-12-03,Out,Injury/Illness - Right Knee; MCL Sprain


In [6]:
def get_game_odds():

    dk_tm_mapping = {
                    "ATL Hawks": "ATL", "BKN Nets": "BRK", "BOS Celtics": "BOS", "CHA Hornets": "CHO", "CHI Bulls": "CHI",
                    "CLE Cavaliers": "CLE", "DAL Mavericks": "DAL", "DEN Nuggets": "DEN", "DET Pistons": "DET", 
                    "GS Warriors": "GSW", "HOU Rockets": "HOU", "IND Pacers": "IND", "LA Clippers": "LAC", 
                    "LA Lakers": "LAL", "MEM Grizzlies": "MEM", "MIA Heat": "MIA", "MIL Bucks": "MIL", "MIN Timberwolves": "MIN",
                    "NO Pelicans": "NOP", "NY Knicks": "NYK", "OKC Thunder": "OKC", "ORL Magic": "ORL", "PHI 76ers": "PHI",
                    "PHO Suns": "PHO", "POR Trail Blazers": "POR", "SA Spurs": "SAS", "SAC Kings": "SAC", "TOR Raptors": "TOR",
                    "UTA Jazz": "UTA", "WAS Wizards": "WAS", "GSW Warriors": "GSW", "LAL Lakers": "LAL", "NOP Pelicans": "NOP",
                    "NYK Knicks": "NYK", "PHX Suns": "PHO", "SAS Spurs": "SAS"
                    }

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
        "Accept": "*/*",
        "Accept-Language": "en-US,en;q=0.9",
        "Referer": "https://sportsbook.draftkings.com/",
        "Origin": "https://sportsbook.draftkings.com"
    }
    response = requests.get(f"https://sportsbook-nash.draftkings.com/sites/US-SB/api/sportscontent/controldata/league/leagueSubcategory/v1/markets?isBatchable=false&templateVars=42648%2C4511&eventsQuery=%24filter%3DleagueId%20eq%20%2742648%27%20AND%20clientMetadata%2FSubcategories%2Fany%28s%3A%20s%2FId%20eq%20%274511%27%29&marketsQuery=%24filter%3DclientMetadata%2FsubCategoryId%20eq%20%274511%27%20AND%20tags%2Fall%28t%3A%20t%20ne%20%27SportcastBetBuilder%27%29&include=Events&entity=events", headers=headers)
    if response.status_code != 200:
        raise Exception('Bad Request')

    ids = []
    rows = []
    for i in range(len(response.json()['selections'])):
        if 'HC' in response.json()['selections'][i]['id'] or 'OU' in response.json()['selections'][i]['id']:
            ids.append(response.json()['selections'][i])
    for i in range(0, len(ids) - 1, 4):
        team1 = ids[i]['label']
        spread1 = ids[i]['points']
        team2 = ids[i+1]['label']
        spread2 = ids[i+1]['points']
        total = ids[i+2]['points']
        rows.append({"Team": team1, "Opp": team2, "Spread": spread1, "Total": total})
        rows.append({"Team": team2, "Opp": team1, "Spread": spread2, "Total": total})
    df_tm_bets = pd.DataFrame(rows)
    
    for key, value in dk_tm_mapping.items():
        df_tm_bets['Team'] = np.where(df_tm_bets.Team == key, value, df_tm_bets.Team)
        df_tm_bets['Opp'] = np.where(df_tm_bets.Opp == key, value, df_tm_bets.Opp)

    df_tm_bets['gm_id'] = df_tm_bets.Team + "_" + df_tm_bets.Opp
    df_tm_bets = df_tm_bets[(df_tm_bets.gm_id.isin(gms_today))].drop(['Opp', 'gm_id'], axis=1)
    
    return df_tm_bets

def get_sportsbook():
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
        "Accept": "*/*",
        "Accept-Language": "en-US,en;q=0.9",
        "Referer": "https://sportsbook.draftkings.com/",
        "Origin": "https://sportsbook.draftkings.com"
    }

    dk_cats = {"PTS": 12488, "AST": 12495, "REB": 12492, "STL": 13508, "BLK": 13780, "STL_BLK": 13781, "TPM": 12497, 
               "PA": 9973, "PR": 9976, "RA": 9974, "PRA": 5001}
    df_lines = pd.DataFrame()
    for key, value in dk_cats.items():
        response = requests.get(f"https://sportsbook-nash.draftkings.com/sites/US-SB/api/sportscontent/controldata/league/leagueSubcategory/v1/markets?isBatchable=false&templateVars=42648%2C{value}&eventsQuery=%24filter%3DleagueId%20eq%20%2742648%27%20AND%20clientMetadata%2FSubcategories%2Fany%28s%3A%20s%2FId%20eq%20%27{value}%27%29&marketsQuery=%24filter%3DclientMetadata%2FsubCategoryId%20eq%20%27{value}%27%20AND%20tags%2Fall%28t%3A%20t%20ne%20%27SportcastBetBuilder%27%29&include=Events&entity=events", headers=headers)
        if response.status_code != 200:
            raise Exception('Bad Request')

        plyr_names = []
        pnt_lines = []
        for i in response.json()['selections']:
            plyr_names.append(i['participants'][0]['name'])
            pnt_lines.append(i['points'])
        df_dk = pd.DataFrame({"Player": plyr_names, f"{key}_line": pnt_lines}).drop_duplicates().reset_index(drop=True)

        if df_lines.empty:
            df_lines = df_dk
        else:
            df_lines = pd.merge(df_lines, df_dk, on="Player", how="outer")

    df_lines = pd.merge(pd.read_csv(f"../tables/{year}/plyr_pos_xref.csv"), df_lines, on='Player', how='right')
    df_lines = pd.merge(df_lines, get_game_odds(), on='Team')
    df_lines.insert(0, 'Date', pd.to_datetime(now))

    partition_save_df(df_lines, f"../tables/{year}/parlay_lines.csv") 
    display(df_lines)

    return df_lines

df_lines = get_sportsbook()

../tables/2025/parlay_lines.csv saved!


,Date,Team,Player,Pos,PTS_line,AST_line,REB_line,STL_line,BLK_line,STL_BLK_line,TPM_line,PA_line,PR_line,RA_line,PRA_line,Spread,Total
0,2025-12-03,MIL,AJ Green,SG,9.5,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,NaN,15.5,4.5,228.5
1,2025-12-03,HOU,Aaron Holiday,PG,10.5,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,NaN,NaN,-15.5,230.5
2,2025-12-03,HOU,Alperen Sengun,C,23.5,6.5,9.5,1.5,0.5,NaN,NaN,29.5,33.5,15.5,39.5,-15.5,230.5
3,2025-12-03,HOU,Amen Thompson,SF,16.5,5.5,7.5,1.5,0.5,NaN,NaN,22.5,24.5,12.5,29.5,-15.5,230.5
4,2025-12-03,IND,Andrew Nembhard,PG,16.5,6.5,2.5,1.5,NaN,NaN,1.5,22.5,18.5,8.5,25.5,7.5,237.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,2025-12-03,ATL,Vit Krejci,PG,8.5,1.5,2.5,NaN,NaN,NaN,1.5,10.5,10.5,4.5,12.5,-3.5,226.5
117,2025-12-03,ORL,Wendell Carter Jr.,C,12.5,NaN,7.5,NaN,0.5,NaN,NaN,NaN,20.5,9.5,NaN,-8.5,235.5
118,2025-12-03,ATL,Zaccharie Risacher,SF,11.5,NaN,2.5,NaN,NaN,NaN,1.5,NaN,14.5,NaN,15.5,-3.5,226.5
119,2025-12-03,SAC,Zach LaVine,SG,18.5,2.5,3.5,0.5,NaN,NaN,2.5,21.5,21.5,5.5,24.5,15.5,230.5


In [7]:
def pick_finder(stat, collect=False):
    df_actuals = pd.read_csv(f"../tables/{year}/parlay_actuals.csv")
    
    df_mtch = df_gms[['AwayABV', 'HomeABV', 'AwayB2B', 'HomeB2B']]
    df_mtch = df_mtch.rename(columns={"AwayABV": "Team", "HomeABV": "Opp", "AwayB2B": "B2B"})[['Team', 'Opp', 'B2B']]
    df_mtch2 = df_mtch.copy().rename(columns={"Team": "Opp", "Opp": "Team", "HomeB2B": "B2B"})[['Team', 'Opp', 'B2B']]
    df_mtch = pd.concat([df_mtch, df_mtch2])
    
    df_rk = con.execute(f"""SELECT *, RANK() OVER (PARTITION BY Pos ORDER BY Off_{stat} DESC) as Off_Rk FROM
                            (SELECT Team, Player, Pos, AVG({stat}) as Off_{stat} FROM df 
                            WHERE Season = 2025 AND {stat} > 0 AND Pos != 'None'
                            AND Team IN ({str(tms_today).replace("[", "").replace("]", "")})
                            GROUP BY Team, Player, Pos)""").fetchdf()
    df_rk_l5 = con.execute(f"""WITH last5 AS (
                                    SELECT Team, Player, Pos, {stat} AS stat_val,
                                    ROW_NUMBER() OVER (PARTITION BY Team, Player ORDER BY Date DESC) AS rn
                                    FROM df
                                    WHERE Season = 2025 AND {stat} > 0 AND Pos != 'None'
                                    AND Team IN ({str(tms_today).replace('[','').replace(']','')})
                            )
                            SELECT Team, Player, Pos, AVG(stat_val) AS Off_L5_{stat},
                            FROM last5
                            WHERE rn <= 5
                            GROUP BY Team, Player, Pos
                            """).fetchdf()

    df_rk = con.execute(f"""SELECT df_rk.Team, df_rk.Player, df_rk.Pos, Off_{stat}, Off_L5_{stat}, Off_Rk FROM df_rk JOIN df_rk_l5 
                            ON df_rk.Team = df_rk_l5.Team AND df_rk.Player = df_rk_l5.Player""").fetchdf()
    df_save = pd.DataFrame()
    for pos in ['PG', 'SG', 'SF', 'PF', 'C']:
        print(pos)
        df_def = con.execute(f"""SELECT Team, '{pos}' AS Pos, Def_{stat}, Def_Rk FROM
                                 (SELECT *, RANK() OVER (ORDER BY Def_{stat}) as Def_Rk FROM
                                 (SELECT Opp as Team, AVG({stat}) as Def_{stat} FROM df 
                                 WHERE MP >= 27 AND Season = 2025 AND Pos = '{pos}'
                                 GROUP BY Opp
                                 ORDER BY AVG(Fpts) DESC))
                                 WHERE Team IN ({str(tms_today).replace("[", "").replace("]", "")})""").fetchdf()

        df_def_l5 = con.execute(f"""WITH last5 AS (
                                    SELECT Opp AS Team, {stat} AS stat_val, 
                                    ROW_NUMBER() OVER (PARTITION BY Opp ORDER BY Date DESC) AS rn
                                    FROM df
                                    WHERE MP >= 27 AND Season = 2025 AND Pos = '{pos}'
                                    )

                                    SELECT Team, '{pos}' AS Pos, AVG(stat_val) AS Def_L5_{stat} FROM last5
                                    WHERE rn <= 5
                                    GROUP BY Team
                                    HAVING Team IN ({str(tms_today).replace('[','').replace(']','')})
                                    """).fetchdf()
        df_def = con.execute(f"""SELECT df_def.Team, df_def.Pos, Def_{stat}, Def_L5_{stat}, Def_Rk FROM df_def 
                               JOIN df_def_l5 ON df_def.Team = df_def_l5.Team""").fetchdf()
        
        # Piece together the current matchups with offensive rankings vs defensive rankings
        df_picks = con.execute(f"""SELECT df_mtch.*, df_rk.* EXCLUDE(Team) FROM df_mtch 
                                  JOIN df_rk ON df_mtch.Team = df_rk.Team
                                  WHERE Pos = '{pos}'""").fetchdf()
        df_picks = con.execute(f"""SELECT df_picks.* EXCLUDE(Opp, Pos), Opp, df_def.Def_{stat}, df_def.Def_L5_{stat}, 
                                   df_def.Def_Rk FROM df_picks 
                                   JOIN df_def ON df_picks.Opp = df_def.Team""").fetchdf()
        df_picks['Rk_Diff'] = df_picks['Def_Rk'] - df_picks['Off_Rk']
        df_picks = con.execute(f"""SELECT t1.* EXCLUDE(Rk_Diff) FROM df_picks t1
                                LEFT JOIN df_inj t2 ON t1.Team = t2.Team AND t1.Player = t2.Player
                                WHERE Status IS NULL
                                ORDER BY Rk_Diff DESC""").fetchdf()
        
        # head to head
        df_h2h = pd.DataFrame()
        for index, row in df_picks.iterrows():
            query = f"""SELECT * EXCLUDE(Gms) FROM
                       (SELECT Player, COUNT(*) as Gms, AVG({stat}) AS AVG_{stat}_H2H FROM df 
                       WHERE Player = ? AND Opp = ? AND Date >= '2023-10-23'
                       GROUP BY Player)
                       WHERE Gms >= 4"""
            df_temp = con.execute(query, [row["Player"], row["Opp"]]).fetchdf()
            df_h2h = pd.concat([df_h2h, df_temp])
        try:
            df_picks = con.execute(f"""SELECT Team, B2B, t1.Player, Off_{stat}, Off_L5_{stat}, t2.AVG_{stat}_H2H, Off_Rk, 
                                   t1.* EXCLUDE(Team, B2B, Player, Off_{stat}, Off_L5_{stat}, Off_Rk)
                                   FROM df_picks t1 LEFT JOIN df_h2h t2 
                                   ON t1.Player = t2.Player""").fetchdf()
        except:
            pass
        
        # Add hit odds (df_actuals)
        df_hit_odds = con.execute(f"""WITH overs AS 
                                        (SELECT Team, Player, SUM({stat}_Diff) as O_Diff, count(*) as Hits FROM df_actuals 
                                        WHERE {stat}_Result = 'O' 
                                        GROUP BY Team, Player)

                                        SELECT overs.Team, overs.Player, Hits, Misses,
                                        O_Diff + U_Diff as {stat}_P_Diff,
                                        hits / (misses + hits) AS {stat}_Hit_Pct, misses / (misses + hits) AS Miss_Pct FROM

                                       (SELECT Team, Player, SUM({stat}_Diff) as U_Diff, count(*) as Misses FROM df_actuals 
                                       WHERE {stat}_Result = 'U' AND {stat}_line IS NOT NULL
                                       GROUP BY Team, Player) unders

                                       JOIN overs ON overs.Team = unders.Team AND overs.Player = unders.Player
                                       ORDER BY {stat}_Hit_Pct DESC""").fetchdf()
        df_picks = con.execute(f"""SELECT df_picks.*, df_hit_odds.{stat}_Hit_Pct, df_hit_odds.{stat}_P_Diff FROM df_picks LEFT JOIN df_hit_odds 
                                   ON df_picks.Team = df_hit_odds.Team AND df_picks.Player = df_hit_odds.Player""").fetchdf()
        
        # Add Spread and Total
        df_picks = con.execute(f"""SELECT df_picks.* EXCLUDE({stat}_Hit_Pct, {stat}_P_Diff), df_lines.{stat}_line, {stat}_Hit_Pct, {stat}_P_Diff, 
                                   Spread, Total FROM df_picks LEFT JOIN df_lines 
                                   ON df_picks.Team = df_lines.Team AND df_picks.Player = df_lines.Player
                                   WHERE {stat}_line IS NOT NULL
                                   ORDER BY Def_Rk DESC""").fetchdf()
        df_save = pd.concat([df_save, df_picks], ignore_index=True)
        if collect == False:
            display(df_picks)
    if collect == True:
        df_save.insert(0, 'Date', pd.to_datetime(now))
        return df_save

In [8]:
for i in categories:
    print(f"==={i}===")
    pick_finder(i)

===PTS===
PG


,Team,B2B,Player,Off_PTS,Off_L5_PTS,AVG_PTS_H2H,Off_Rk,Opp,Def_PTS,Def_L5_PTS,Def_Rk,PTS_line,PTS_Hit_Pct,PTS_P_Diff,Spread,Total
0,CLE,1,Lonzo Ball,6.714286,6.600000,NaN,34,POR,24.066667,29.6,30,6.5,NaN,NaN,-11.5,240.5
1,CLE,1,Darius Garland,14.714286,15.000000,NaN,15,POR,24.066667,29.6,30,18.5,0.500000,-2.0,-11.5,240.5
2,SAS,1,De'Aaron Fox,24.090909,25.800000,NaN,4,ORL,23.750000,24.0,28,25.5,0.750000,-5.0,8.5,235.5
3,HOU,0,Aaron Holiday,9.909091,11.600000,5.333333,28,SAC,23.076923,27.2,27,10.5,NaN,NaN,-15.5,230.5
4,HOU,0,Reed Sheppard,13.941176,16.600000,NaN,16,SAC,23.076923,27.2,27,14.5,0.333333,-2.5,-15.5,230.5
5,DAL,0,Cooper Flagg,17.047619,19.200000,NaN,13,MIA,22.857143,22.6,26,17.5,0.500000,26.0,4.5,238.5
6,DET,0,Cade Cunningham,28.222222,30.400000,25.857143,2,MIL,22.833333,32.6,25,27.5,0.600000,1.5,-4.5,228.5
7,ATL,0,Vit Krejci,11.055556,6.600000,12.000000,24,LAC,22.277778,22.8,22,8.5,0.400000,-3.5,-3.5,226.5
8,ORL,1,Anthony Black,13.428571,18.800000,6.500000,19,SAS,21.625000,18.8,19,13.5,NaN,NaN,-8.5,235.5
9,ORL,1,Jalen Suggs,13.437500,15.400000,NaN,18,SAS,21.625000,18.8,19,16.5,0.250000,-11.0,-8.5,235.5


SG


,Team,B2B,Player,Off_PTS,Off_L5_PTS,AVG_PTS_H2H,Off_Rk,Opp,Def_PTS,Def_L5_PTS,Def_Rk,PTS_line,PTS_Hit_Pct,PTS_P_Diff,Spread,Total
0,CHO,0,Sion James,6.850000,5.40,NaN,40,NYK,19.777778,23.4,28,6.5,0.666667,4.5,8.5,235.5
1,CHO,0,Collin Sexton,16.100000,18.20,22.600000,10,NYK,19.777778,23.4,28,14.5,NaN,NaN,8.5,235.5
2,CLE,1,Jaylon Tyson,12.411765,15.00,NaN,16,POR,19.210526,17.6,27,11.5,0.833333,28.0,-11.5,240.5
3,CLE,1,Donovan Mitchell,30.600000,31.40,NaN,1,POR,19.210526,17.6,27,28.5,0.666667,16.0,-11.5,240.5
4,POR,1,Shaedon Sharpe,20.875000,20.40,19.000000,6,CLE,18.750000,16.4,25,20.5,NaN,NaN,11.5,240.5
5,ORL,1,Desmond Bane,19.190476,26.80,23.250000,9,SAS,18.529412,19.4,24,23.5,0.400000,17.5,-8.5,235.5
6,MIA,0,Tyler Herro,24.750000,24.75,25.750000,3,DAL,18.294118,18.6,22,22.5,NaN,NaN,-4.5,238.5
7,MIA,0,Pelle Larsson,10.150000,7.00,NaN,25,DAL,18.294118,18.6,22,8.5,0.500000,0.0,-4.5,238.5
8,DEN,0,Bruce Brown,6.684211,5.00,9.000000,44,IND,18.086957,21.4,21,9.5,0.400000,-10.5,-7.5,237.5
9,DEN,0,Tim Hardaway Jr.,11.950000,14.40,12.285714,19,IND,18.086957,21.4,21,13.5,0.800000,5.5,-7.5,237.5


SF


...

,Team,B2B,Player,Off_STL_BLK,Off_L5_STL_BLK,AVG_STL_BLK_H2H,Off_Rk,Opp,Def_STL_BLK,Def_L5_STL_BLK,Def_Rk,STL_BLK_line,STL_BLK_Hit_Pct,STL_BLK_P_Diff,Spread,Total
0,DET,0,Ausar Thompson,2.437500,1.4,0.857143,3,MIL,2.043478,2.2,24,2.5,NaN,NaN,-4.5,228.5
1,MIA,0,Andrew Wiggins,2.166667,2.4,1.428571,12,DAL,1.931034,1.2,20,1.5,NaN,NaN,-4.5,238.5
2,BRK,0,Michael Porter Jr.,1.416667,1.8,1.750000,34,CHI,1.916667,2.2,19,1.5,0.666667,-0.5,7.5,232.5
3,ORL,1,Franz Wagner,1.882353,2.0,1.000000,18,SAS,1.346154,1.4,6,1.5,0.400000,0.5,-8.5,235.5


PF


,Team,B2B,Player,Off_STL_BLK,Off_L5_STL_BLK,AVG_STL_BLK_H2H,Off_Rk,Opp,Def_STL_BLK,Def_L5_STL_BLK,Def_Rk,STL_BLK_line,STL_BLK_Hit_Pct,STL_BLK_P_Diff,Spread,Total
0,CLE,1,Evan Mobley,2.842105,3.0,2.750000,3,POR,2.437500,2.4,28,2.5,0.600000,2.5,-11.5,240.5
1,MIL,0,Giannis Antetokounmpo,2.133333,1.4,3.000000,13,DET,2.133333,2.2,22,2.5,NaN,NaN,4.5,228.5
2,IND,0,Pascal Siakam,2.250000,2.8,0.000000,7,DEN,2.105263,2.2,21,1.5,0.500000,2.0,7.5,237.5
3,CHI,0,Matas Buzelis,2.500000,3.2,NaN,5,BRK,2.000000,2.2,20,2.5,0.500000,2.0,-7.5,232.5
4,CHO,0,Miles Bridges,1.529412,1.4,1.800000,31,NYK,1.600000,1.2,11,1.5,0.333333,-0.5,8.5,235.5
5,DAL,0,P.J. Washington,2.157895,2.6,1.333333,11,MIA,1.461538,0.6,5,1.5,0.750000,2.0,4.5,238.5
6,DAL,0,Anthony Davis,2.857143,2.6,3.500000,2,MIA,1.461538,0.6,5,2.5,NaN,NaN,4.5,238.5


C


,Team,B2B,Player,Off_STL_BLK,Off_L5_STL_BLK,AVG_STL_BLK_H2H,Off_Rk,Opp,Def_STL_BLK,Def_L5_STL_BLK,Def_Rk,STL_BLK_line,STL_BLK_Hit_Pct,STL_BLK_P_Diff,Spread,Total
0,CHI,0,Nikola Vucevic,1.500000,1.4,1.000000,36,BRK,2.363636,1.8,20,1.5,0.2,-3.5,-7.5,232.5
1,MIA,0,Bam Adebayo,2.333333,2.4,1.200000,18,DAL,2.166667,2.8,16,1.5,NaN,NaN,-4.5,238.5
2,NYK,0,Karl-Anthony Towns,2.000000,1.6,0.833333,25,CHO,1.533333,1.2,2,1.5,0.5,0.0,-8.5,235.5


In [9]:
df_stats = pd.DataFrame()
for i in categories:
    print(f"==={i}===")
    df_temp = pick_finder(i, collect=True)
    df_stats = pd.concat([df_stats, df_temp], ignore_index=True)
    
df_stats = con.execute("SELECT Date, Team, B2B, Spread, Total, Player, Opp, * EXCLUDE(Date, Team, B2B, Player, Opp, Off_Rk, Def_Rk, Spread, Total) FROM df_stats").fetchdf()
df_save = pd.DataFrame()
for player in df_stats.Player.unique():
    df_temp = df_stats[(df_stats.Player == player)]
    for col in df_temp.columns.difference(['Date', 'Team', 'Player']):
        df_temp[col] = df_temp[col].ffill()
        df_temp[col] = df_temp[col].backfill()
    df_temp = df_temp.drop_duplicates()
    df_save = pd.concat([df_save, df_temp])
print('Saving for Date:', now)
partition_save_df(df_save, f"../tables/{year}/parlay_stats.csv")
display(df_save)

===PTS===
PG
SG
SF
PF
C
===AST===
PG
SG
SF
PF
C
===REB===
PG
SG
SF
PF
C
===PR===
PG
SG
SF
PF
C
===PA===
PG
SG
SF
PF
C
===RA===
PG
SG
SF
PF
C
===PRA===
PG
SG
SF
PF
C
===TPM===
PG
SG
SF
PF
C
===STL===
PG
SG
SF
PF
C
===BLK===
PG
SG
SF
PF
C
===STL_BLK===
PG
SG
SF
PF
C
Saving for Date: 2025-12-02
../tables/2025/parlay_stats.csv saved!


""
